In [1]:
# !pip install selenium
# !apt-get update # to update ubuntu to correctly run apt install
# !apt install chromium-chromedriver
# !cp /usr/lib/chromium-browser/chromedriver /usr/bin
import sys
sys.path.insert(0,'/usr/lib/chromium-browser/chromedriver')
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.common.exceptions import TimeoutException, NoSuchElementException
from selenium.webdriver.support import expected_conditions as EC
import time
from bs4 import BeautifulSoup
import re

In [2]:
# from google.colab import drive
# drive.mount('/content/drive')
from threading import Thread
from datetime import date, timedelta

In [ ]:
!top

=top - 16:30:40 up 3 min,  0 users,  load average: 0.31, 0.41, 0.19
Tasks:  11 total,   1 running,   9 sleeping,   0 stopped,   1 zombie
%Cpu(s): 10.8 us,  5.0 sy,  0.0 ni, 80.8 id,  3.2 wa,  0.0 hi,  0.1 si,  0.0 st
KiB Mem : 13302920 total, 10838872 free,   587372 used,  1876676 buff/cache
KiB Swap:        0 total,        0 free,        0 used. 12479644 avail Mem 

    PID USER      PR  NI    VIRT    RES    SHR S  %CPU %MEM     TIME+ COMMAND   
      1 root      20   0     992      4      0 S   0.0  0.0   0:00.56 docker-i+ 
      7 root      20   0  339816  50640  32720 S   0.0  0.4   0:00.76 node      
     18 root      20   0   35892   4732   3628 S   0.0  0.0   0:00.47 tail      
     28 root      20   0       0      0      0 Z   0.0  0.0   0:06.99 python3   
     29 root      20   0  160408  41408  13060 S   0.0  0.3   0:00.60 colab-fi+ 
     42 root      20   0  203828  60268  15612 S   0.0  0.5   0:01.51 jupyter-+ 
     43 root      20   0  707076   5572   4312 S   0.0  0.0   

In [3]:
prefix_link = 'https://coinmarketcap.com/historical/'
process_count = 5
last_index = [0 for _ in range(process_count)]

def getDriver():
    chrome_options = webdriver.ChromeOptions()
    chrome_options.add_argument('--headless')
    chrome_options.add_argument('--no-sandbox')
    chrome_options.add_argument('--disable-dev-shm-usage')
    driver = webdriver.Chrome(executable_path="D:\\USERS\\szhu76\\Downloads\\chromedriver_win32_2\\chromedriver",chrome_options=chrome_options)
    return driver

def driverLink(driver, date_str):
    print("inside driver link")
    link = prefix_link + date_str
    driver.get(link)
    time.sleep(2)

def scrape_record(row):
    rank = name = symbol = mcap = price = csupply = volume = h1 = h24 = d7 = None
    header = "Rank,Name,Symbol,Market Cap,Price,Circulating Supply,Volume,%1h,%24h,%7d"
    cells = row.find_all('td')
    name_class = 'cmc-table__column-name--name cmc-link'
    mcap_class = 'sc-1ow4cwt-1 ieFnWP'
    if len(cells) > 3:
        # Rank
        if cells[0].find('div'): rank = cells[0].find('div').text
        # Name
        if cells[1].find(class_=re.compile(name_class)):
          name = cells[1].find(class_=re.compile(name_class)).text
        # Symbol
        if cells[2].find('div'): symbol = cells[2].find('div').text
        # Market cap
        if cells[3].find(class_=re.compile(mcap_class)): 
          mcap = cells[3].find(class_=re.compile(mcap_class)).text.replace(",", "").replace("$","")
        # Price
        if cells[4].find('div'): price = cells[4].find('div').text.replace(",", "").replace("$","")
        # Circulating Supply
        if cells[5].find('div'): csupply = cells[5].find('div').text.replace(",", "")
        # Volume
        if cells[6].find('a'): volume = cells[6].find('a').text.replace(",", "").replace("$","")
        # %1h
        if cells[7].find('div'): h1 = cells[7].find('div').text.replace("%","")
        # %24h
        if cells[8].find('div'): h24 = cells[8].find('div').text.replace("%","")
        # %7d
        if cells[9].find('div'): d7 = cells[9].find('div').text.replace("%","")
    else:
        return ""
    return "{},{},{},{},{},{},{},{},{},{}".format(rank, name, symbol, mcap,price, csupply,volume,h1,h24,d7)


def scrapePage(driver, output_file, p):
    with open(output_file, 'a', encoding="utf-8") as file:
        global last_index
        html = driver.page_source
        soup = BeautifulSoup(html, 'lxml')
        rows = soup.find_all(class_=re.compile('cmc-table-row'))
        start = last_index[p]
        for ri in range(start, len(rows)):
            row = rows[ri]
            last_index[p] += 1
            ret = scrape_record(row)
            if len(ret) > 0:
                file.write(ret + '\n')
            else:
                last_index[p] -= 1


def scroll_to_bottom(driver, cur_page, output_file, p):
    parts = 10
    SCROLL_PAUSE_TIME = 1
    for i in range(parts+1):
        driver.execute_script("window.scrollTo(0, document.body.scrollHeight * " + str((cur_page-1)/cur_page + i/(cur_page * parts)) + ");")
        # Wait to load page
        time.sleep(SCROLL_PAUSE_TIME)
        WebDriverWait(driver, SCROLL_PAUSE_TIME)
        if i > 0:
            scrapePage(driver, output_file, p)


def scrape(driver, output_file, p):
    print("starting to scrape")
    scroll_to_bottom(driver, 1, output_file, p)
    explore = True
    if explore:
        wait = WebDriverWait(driver, 5)

        try:
            wait.until(EC.element_to_be_clickable((By.LINK_TEXT, "Load More")))
        except TimeoutException as e:
            print(e)

        button = driver.find_element_by_xpath('//button[normalize-space()="Load More"]')
        load_more = True
        pages = 1
        #while load_more:
        while pages <= 10:
            button.click()
            print("Clicked. Page #", pages)
            wait = WebDriverWait(driver, 10)
            scroll_to_bottom(driver, pages, output_file, p)
            pages += 1
            wait = WebDriverWait(driver, 10)
            try:
                button = driver.find_element_by_xpath('//button[normalize-space()="Load More"]')
            except NoSuchElementException as e2:  # Load More button is no longer present;
                load_more = False

        print(str(pages) + " pages found.")


def get_dates(sdate, edate):
    return [(sdate+timedelta(days=x)).strftime("%Y%m%d") for x in range((edate-sdate).days)]

def driver_prog(process_dates, p):
  for d in process_dates:
      drivers[p] = getDriver()
      global last_index
      last_index[p] = 0
      print("got driver, thread", p)
      print("Processing date ", d, " thread ", p, len(drivers))
      driverLink(drivers[p], d)
      output_file = path_prefix + d + ".csv"
      print(output_file)
      scrape(drivers[p], output_file, p)
      drivers[p].close()

path_prefix = "D:\\Shuai-Jingbo-Pei-yu\\CoinMarketCap_CoinPrice_data\\Coin_real_price\\Coinmarket data"
drivers = [None for _ in range(process_count)]
start_date = date(2022,9,20)
end_date = date.today()
print(start_date, end_date)

dates = get_dates(start_date, end_date)  # List of dates in desired string format 
print(dates)
threads = []
#driver_prog(dates, 0)
for p in range(process_count):
  arg = [dates[i] for i in range(len(dates)) if i%process_count == p]
  t = Thread(target=driver_prog, args=(arg,p))
  t.start()
  threads.append(t)

for p in range(process_count):
  t = threads[p]
  t.join()
  #proc = multiprocessing.Process(target=driver_prog, args=(path_prefix,arg))
  #proc.start()


2022-02-18 2022-06-06
['20220218', '20220219', '20220220', '20220221', '20220222', '20220223', '20220224', '20220225', '20220226', '20220227', '20220228', '20220301', '20220302', '20220303', '20220304', '20220305', '20220306', '20220307', '20220308', '20220309', '20220310', '20220311', '20220312', '20220313', '20220314', '20220315', '20220316', '20220317', '20220318', '20220319', '20220320', '20220321', '20220322', '20220323', '20220324', '20220325', '20220326', '20220327', '20220328', '20220329', '20220330', '20220331', '20220401', '20220402', '20220403', '20220404', '20220405', '20220406', '20220407', '20220408', '20220409', '20220410', '20220411', '20220412', '20220413', '20220414', '20220415', '20220416', '20220417', '20220418', '20220419', '20220420', '20220421', '20220422', '20220423', '20220424', '20220425', '20220426', '20220427', '20220428', '20220429', '20220430', '20220501', '20220502', '20220503', '20220504', '20220505', '20220506', '20220507', '20220508', '20220509', '2022

C:\Users\szhu76\AppData\Local\Temp\40\ipykernel_355740\4242525020.py:10: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(executable_path="D:\\USERS\\szhu76\\Downloads\\chromedriver_win32_2\\chromedriver",chrome_options=chrome_options)
C:\Users\szhu76\AppData\Local\Temp\40\ipykernel_355740\4242525020.py:10: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(executable_path="D:\\USERS\\szhu76\\Downloads\\chromedriver_win32_2\\chromedriver",chrome_options=chrome_options)


got driver, threadgot driver, thread 4
Processing date  20220222  thread  4 5
inside driver link
got driver, thread 0
Processing date  20220218  thread  0 5
inside driver link
 1
Processing date  20220219  thread  1 5
inside driver link
got driver, thread 3
Processing date  20220221  thread  3 5
inside driver link
got driver, thread 2
Processing date  20220220  thread  2 5
inside driver link
D:\Shuai-Jingbo-Pei-yu\CoinMarketCap_CoinPrice_data\Coin_real_price\Coinmarket data20220218.csv
starting to scrape
D:\Shuai-Jingbo-Pei-yu\CoinMarketCap_CoinPrice_data\Coin_real_price\Coinmarket data20220221.csv
starting to scrape
D:\Shuai-Jingbo-Pei-yu\CoinMarketCap_CoinPrice_data\Coin_real_price\Coinmarket data20220222.csv
starting to scrape
D:\Shuai-Jingbo-Pei-yu\CoinMarketCap_CoinPrice_data\Coin_real_price\Coinmarket data20220220.csv
starting to scrape
D:\Shuai-Jingbo-Pei-yu\CoinMarketCap_CoinPrice_data\Coin_real_price\Coinmarket data20220219.csv
starting to scrape
Message: 
Stacktrace:
Backtrac

C:\Users\szhu76\AppData\Local\Temp\40\ipykernel_355740\4242525020.py:94: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  button = driver.find_element_by_xpath('//button[normalize-space()="Load More"]')


Clicked. Page # 1
Clicked. Page # 1
Message: 
Stacktrace:
Backtrace:
	Ordinal0 [0x01295843+2512963]
	Ordinal0 [0x0122EED1+2092753]
	Ordinal0 [0x01131F48+1056584]
	Ordinal0 [0x0115C70B+1230603]
	Ordinal0 [0x01186442+1401922]
	Ordinal0 [0x0117562A+1332778]
	Ordinal0 [0x0118487F+1394815]
	Ordinal0 [0x011754BB+1332411]
	Ordinal0 [0x01151D14+1187092]
	Ordinal0 [0x01152B79+1190777]
	GetHandleVerifier [0x01410C56+1501718]
	GetHandleVerifier [0x014BE97F+2213695]
	GetHandleVerifier [0x01318C92+485970]
	GetHandleVerifier [0x01317D59+482073]
	Ordinal0 [0x0123471D+2115357]
	Ordinal0 [0x01238678+2131576]
	Ordinal0 [0x012387B2+2131890]
	Ordinal0 [0x01241971+2169201]
	BaseThreadInitThunk [0x758E0419+25]
	RtlGetAppContainerNamedObjectPath [0x77C074ED+237]
	RtlGetAppContainerNamedObjectPath [0x77C074BD+189]

Clicked. Page # 1
Message: 
Stacktrace:
Backtrace:
	Ordinal0 [0x01295843+2512963]
	Ordinal0 [0x0122EED1+2092753]
	Ordinal0 [0x01131F48+1056584]
	Ordinal0 [0x0115C70B+1230603]
	Ordinal0 [0x01186442+

C:\Users\szhu76\AppData\Local\Temp\40\ipykernel_355740\4242525020.py:106: DeprecationWarning: find_element_by_xpath is deprecated. Please use find_element(by=By.XPATH, value=xpath) instead
  button = driver.find_element_by_xpath('//button[normalize-space()="Load More"]')


Clicked. Page # 2
Clicked. Page # 2
Clicked. Page # 2
Clicked. Page # 2
Clicked. Page # 2
Clicked. Page # 3
Clicked. Page # 3
Clicked. Page # 3
Clicked. Page # 3
Clicked. Page # 3
Clicked. Page # 4
Clicked. Page # 4
Clicked. Page # 4
Clicked. Page # 4
Clicked. Page # 5
Clicked. Page # 5
Clicked. Page # 5
Clicked. Page # 4
Clicked. Page # 6
Clicked. Page # 6
Clicked. Page # 5
Clicked. Page # 6
Clicked. Page # 7
Clicked. Page # 7
Clicked. Page # 5
Clicked. Page # 6
Clicked. Page # 7
Clicked. Page # 8
Clicked. Page # 8
Clicked. Page # 7
Clicked. Page # 8
Clicked. Page # 6
Clicked. Page # 9
Clicked. Page # 9
Clicked. Page # 8
Clicked. Page # 9
Clicked. Page # 10
Clicked. Page # 10
Clicked. Page # 9
Clicked. Page # 10
11 pages found.
got driver, thread 1
Processing date  20220224  thread  1 5
inside driver link
11 pages found.
Clicked. Page # 7
D:\Shuai-Jingbo-Pei-yu\CoinMarketCap_CoinPrice_data\Coin_real_price\Coinmarket data20220224.csv
starting to scrape
got driver, thread 0
Processing d